In [ ]:
# import required packages
import tensorflow as tf 
import cv2
import os
import matplotlib.pyplot as plt
import numpy as np

# Live Video Stream without detection model

In [ ]:
# read the cascade classifiers for face, eyes and mouth
faceCascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
eyeCascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_eye.xml")
smileCascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_smile.xml")

cap = cv2.VideoCapture(-1) 
# generally -1 works on Ubuntu, 0 works on Windows
if not cap.isOpened():
    cap = cv2.VideoCapture(0)
if not cap.isOpened():
    raise IOError("Cannot open webcam")

    
while True:
    # grab the frame from the video stream, resize it, and convert it to grayscale
    ret,frame = cap.read()
    frame = cv2.resize(frame, None, fx=1, fy=1, interpolation=cv2.INTER_AREA)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # find and mark face 
    faces = faceCascade.detectMultiScale(gray, 1.1, 4, minSize=(165, 165))    
    for x,y,w,h in faces:
        roi_gray = gray[y:y+h, x:x+h]
        roi_color = frame[y:y+h, x:x+w]
        cv2.rectangle(frame, (x,y), (x+w, y+h), (255, 0, 0), 2)

    # find and mark the smile, especially the mouth and prevent missmarked eyes 
    smiles = smileCascade.detectMultiScale(gray, 1.7, 11)
    for (sx,sy,sw,sh) in smiles:
        cv2.rectangle(frame,(sx,sy),(sx+sw,sy+sh),(0, 0, 255),2)
        
    # find and mark eyes
    eyes = eyeCascade.detectMultiScale(gray, 1.1, 4, minSize=(67, 67))
    for (ex,ey,ew,eh) in eyes:
        cv2.rectangle(frame,(ex,ey),(ex+ew,ey+eh),(0,255,0),2)        

    # show processed live video stream 
    cv2.imshow("Demo", frame)
    
    # q for exit
    if cv2.waitKey(2) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()

# Live Video Stream and Drowsiness Detection via Our Model

In [ ]:
# load the trained model
new_model = tf.keras.models.load_model(r'/home/eren/drowsiness/finalModel.h5')
# read the cascade classifiers for face, eyes and mouth
faceCascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
eyeCascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_eye.xml")
smileCascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_smile.xml")

# alarm is set
from pygame import mixer
mixer.init()
sound= mixer.Sound('/home/eren/drowsiness/alarm.wav')

cap = cv2.VideoCapture(-1)
# generally -1 works on Ubuntu, 0 works on Windows
if not cap.isOpened():
    cap = cv2.VideoCapture(0)
if not cap.isOpened():
    raise IOError("Cannot open webcam")

# drowsiness detection parameter, can be changed for faster or slower response
counter = 0   
while True:
    # grab the frame from the video stream, resize it, and convert it to grayscale
    ret,frame = cap.read()
    height,width = frame.shape[0:2]
    frame = cv2.resize(frame, None, fx=1, fy=1, interpolation=cv2.INTER_AREA)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # find and mark face
    faces = faceCascade.detectMultiScale(gray, 1.1, 4, minSize=(165, 165))    
    for x,y,w,h in faces:
        roi_gray = gray[y:y+h, x:x+h]
        roi_color = frame[y:y+h, x:x+w]
        cv2.rectangle(frame, (x,y), (x+w, y+h), (255, 0, 0), 2)

    # find and mark the smile, especially the mouth and prevent missmarked eyes 
    smiles = smileCascade.detectMultiScale(gray, 1.7, 11)
    #for (sx,sy,sw,sh) in smiles:
        #cv2.rectangle(frame,(sx,sy),(sx+sw,sy+sh),(0, 0, 255),2)
        # no need to show
        
    # find and mark eyes
    eyes = eyeCascade.detectMultiScale(gray, 1.1, 4, minSize=(65, 65))
    for (ex,ey,ew,eh) in eyes:
        cv2.rectangle(frame,(ex,ey),(ex+ew,ey+eh),(0,255,0),2)        
        eyes_roi_gray = gray[ey:ey+eh, ex:ex+ew]
        eyes_roi_color = frame[ey:ey+eh, ex:ex+ew]
            
        eyess = eyeCascade.detectMultiScale(eyes_roi_gray)
        if len(eyess) == 0:
            print("eyes are not detected")
        else:
            for(mex,mey,mew,meh) in eyess: # model eye x,y,w,h
                eyes_roi = eyes_roi_color[mey: mey+meh, mex:mex +mew]

        # load cropped and grayscaled eye frames into model
        final_image = cv2.resize(eyes_roi, (224,224))
        final_image = np.expand_dims(final_image, axis = 0) #4th dimension is needed
        final_image = final_image/255.0
        Predictions = new_model.predict(final_image)
        # show eye openness ratio and drowsiness muchness   
        print(Predictions)
        print(counter)
        cv2.putText(frame,'parameter:'+str(counter),(10,height-50),fontFace=cv2.FONT_HERSHEY_COMPLEX_SMALL,fontScale=1,color=(255,255,255),thickness=1,lineType=cv2.LINE_AA)
        
        #decide whether eyes are open or closed then act
        if(Predictions>0.57):
            counter = counter - 3
            status = "Open Eyes"
            font = cv2.FONT_HERSHEY_SIMPLEX
            cv2.putText(frame, status, (110,150), font, 3, (255, 0, 0), 2, cv2.LINE_AA)
            cv2.putText(frame,"Active",(10,height-20),fontFace=cv2.FONT_HERSHEY_COMPLEX_SMALL,fontScale=1,color=(255, 0, 0),thickness=1,lineType=cv2.LINE_AA) 
            if (counter<0):
                counter=0
                        
        elif(Predictions)<0.47:  
            counter = counter + 2
            status = "Closed Eyes"
            cv2.putText(frame, status, (110,150), font, 3, (0, 0, 255), 2, cv2.LINE_AA)
            cv2.rectangle(frame, (x,y), (x+w, y+h), (0, 0, 255), 2)
            if counter > 15:
                cv2.putText(frame,"WARNING:Drowsiness Detect!",(10,height-20),fontFace=cv2.FONT_HERSHEY_COMPLEX_SMALL,fontScale=1,color=(0, 0, 255),
                        thickness=1,lineType=cv2.LINE_AA)
                try:
                    # turn on alarm and wake the driver
                    sound.play()
                except:
                    pass
                
            # if counter > 30:   try: "vibrate the steering wheel and seat with actuators"  (for later versions)          
        
    # show processed live video stream 
    cv2.imshow("EHB 420E-Drowsiness Detection Project", frame)
        
    # q for exit
    if cv2.waitKey(2) & 0xFF ==ord("q"):
        break
           
cap.release()
cv2.destroyAllWindows()